In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2019)

import nltk
nltk.download('wordnet')


In [93]:
questions = pd.read_csv('PostQuestionsFiltered.tsv', delimiter='\t', encoding='utf-8', nrows=1000)

ques_title = questions[['body']]

ques_title['index'] = ques_title.index

titles = ques_title

'boostcompilationconfigurerhelgraph-tool'

In [98]:
print(titles[:5])

                                                body  index
0  <p>I need to use Python's graph-tool on a RHEL...      0
1  <p>I need a HashMap, which the key is String a...      1
2  <p>I've to make a map with multiple directions...      2
3  <p>I'm working on a project using React for my...      3
4  <p>I wrote a simple program to monitor my ping...      4


In [100]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [101]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
title_sample = titles[titles['index'] == 90].values[0][0]

words = []
for word in title_sample.split(' '):
    words.append(word)

processed_title = titles['body'].map(preprocess)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_title]

In [109]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [113]:
for index, score in sorted(lda_model[bow_corpus[90]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7287601232528687	 
Topic: 0.039*"valu" + 0.025*"data" + 0.022*"line" + 0.018*"file" + 0.017*"function" + 0.016*"http" + 0.014*"map" + 0.013*"googl" + 0.013*"marker" + 0.012*"point"

Score: 0.19449633359909058	 
Topic: 0.017*"applic" + 0.015*"public" + 0.015*"class" + 0.014*"string" + 0.014*"googl" + 0.012*"null" + 0.012*"error" + 0.012*"bitmap" + 0.011*"float" + 0.011*"function"

Score: 0.06629209965467453	 
Topic: 0.024*"function" + 0.021*"class" + 0.015*"text" + 0.015*"height" + 0.015*"info" + 0.014*"attr" + 0.013*"margin" + 0.013*"style" + 0.013*"font" + 0.013*"return"
